In [ ]:
pickle_file: str = None

In [ ]:
from pathlib import Path
import pickle
from IPython.display import display, HTML

with open(pickle_file, "rb") as f:
    parameters = pickle.load(f)
X = parameters["X"]
y = parameters["y"]
y_cv_pred = parameters["y_cv_pred"]
model_path: Path = parameters["model_path"]
model_config = parameters["model_config"]
mean_test_accuracy = parameters["mean_test_accuracy"]
threshold_waypoints = parameters["threshold_waypoints"]
training_dir = parameters["training_dir"]

# Document Quality Analyzer training results

Your trained model has been saved in your training directory as:

In [ ]:
print(model_path.parts[-1])

Please provide the contents of this file in the configuration when instantiating the ScanbotSDK Document Quality Analyzer.

In [ ]:
html = f"""
<script>
model_config = "{model_config}";
"""+"""
function copy_model_config_to_clipboard(e){
if(navigator.clipboard){
  navigator.clipboard.writeText(model_config).then(function() {
    e.target.innerText = 'Copied model config to your clipboard!';
  }, function(err) {
    alert('Could not copy model configuration to clipboard: ', err);
  });
} else {
    alert('Could not copy model configuration to clipboard.');
}
}
</script>
<button onclick="copy_model_config_to_clipboard(event)">Click here to copy the model config to your clipboard</button>
"""
display(HTML(html))


## Model quality

In [ ]:
display(HTML(f"Your model achieved an mean cross-validation accuracy of <b>{mean_test_accuracy * 100:.1f}%</b>."))

The cross-validation accuracy describes how well the model was able to learn from your dataset.
* &lt;80% means that the good and bad samples in your dataset are hard to distinguish.
* &gt;80% means that your dataset is learnable but needs more samples.
* &gt;90% is typically good enough for production.

During training, the model had problems to correctly classify the following samples:

In [ ]:
false_positives = ((y == 0) & (y_cv_pred > 0))
false_negatives = ((y == 1) & (y_cv_pred < 0))

def link_sample(sample):
    relative_path = Path(sample["image_path"]).relative_to(training_dir)
    return f'<a href="{relative_path}">{relative_path}</a><br>\n'

html = f'<details><summary>Click here to show the false positives ("unacceptable" training samples classified as "acceptable", {false_positives.sum()} of {(y == 0).sum()} samples).</summary>'
for i, x in X[false_positives].iterrows():
    html += link_sample(x)
html += '</details>'
html += f'<details><summary>Click here to show the false negatives ("acceptable" training samples classified as "unacceptable", {false_negatives.sum()} of {(y == 1).sum()} samples).</summary>'
for i, x in X[false_negatives].iterrows():
    html += link_sample(x)
html += '</details>'
display(HTML(html))

## Recommended next Steps

**If your model's cross validation accuracy (see above) is less than 90%:**<br>
Add more good and bad examples to improve model accuracy, or consider removing confusing examples (see list of confusing examples above).

**If your model's cross validation accuracy is greater than 90%:**<br>
* Copy the model config string (see above) into your app's configuration. Please refer to the [Scanbot SDK Documentation](https://docs.scanbot.io/) for details.
* Optionally, you can choose a non-default uncertainty threshold. See the following section for details.



## Choosing an uncertainty threshold
Documents which are neither clearly good nor clearly bad may be classified incorrectly.
A borderline good document may be classified as unacceptable and a borderline bad document may be classified as acceptable.
An incorrect classification often leads to user frustration and generally should be avoided.

DoQA is able to put borderline documents into a third category: uncertain. The third category provides an alternative workflow path for your users:
* If a document's quality is acceptable, then you know it's definitely acceptable: accept document for further processing
* If a document's quality is unacceptable, then you know it's definitely unacceptable: request that user retake the photo, don't allow the user to skip the quality check.
* If a document's quality is uncertain, then you're unsure: request that the user retake the photo only once, then allow the user to skip the quality check, if the second photo also has uncertain quality.

The "Uncertainty Threshold" parameter lets you decide how certain the model should be.
A higher threshold will produce fewer incorrect classifications, but more uncertain ones. By default it is set to 0.5.
A lower threshold will produce fewer uncertain classifications, but more incorrect ones. <br>
Every dataset is different, so every dataset's optimal Uncertainty Threshold is different.
Additionally, every use case's tolerance for incorrect classifications is different.
Look at the graphs and tables below to pick the optimal Uncertainty Threshold for your dataset and use case.

The following plots show what performance your can expect from your model.<br>
The x-axis display the Uncertainty Threshold.
The y-axis show what percentage of the documents get correctly / incorrectly classified.

In [ ]:
from train_plots import plot_classification
from configurator_utils import colorblind_friendly_plots

fig1 = plot_classification(
    y=y,
    y_pred=y_cv_pred,
    threshold_waypoints=threshold_waypoints,
    output_dir=None,
    colorblind_friendly=False
)
fig2 = plot_classification(
    y=y,
    y_pred=y_cv_pred,
    threshold_waypoints=threshold_waypoints,
    output_dir=None,
    colorblind_friendly=True
)
display(HTML(colorblind_friendly_plots(fig1.to_html(full_html=False), fig2.to_html(include_plotlyjs=False, full_html=False))))

In [ ]:
plot_svg = Path('reference_classification_uncertainty_plot.svg').read_text()
display(HTML(f'<details><summary>Click here to show how the plot of how a successful training might look like.<br>'
             f'If your training plot looks significantly different, there might be an issue with your training images.'
             f'</summary>{plot_svg}</details>'))